# OOP of BlockChain Project

In [ ]:
# imports
from solcx import compile_standard, install_solc
import json
from web3 import Web3
from dotenv import load_dotenv, find_dotenv
import os
import hashlib

In [ ]:
# setup
# load from .env file
load_dotenv(find_dotenv())
install_solc('0.6.0')

In [ ]:
class ContractCreator:
    
    def __init__(self, file_path):
        self.contract_location = file_path
        # generate name for compiled code JSON file
        self.compiled_contract_location = self.contract_location[:self.contract_location.index('.sol')] + '.json'
        # blockchain connection and credentials:
        self.w3 = Web3(
                Web3.HTTPProvider("HTTP://127.0.0.1:7545")
            )  # Get this address from RPC provider in ganache GUI
        self.chain_id = 1337  # From Network ID of ganache GUI
        self.my_address = os.getenv("PUBLIC_KEY")  # address to deploy from - one of fake accounts in GUI
        self.private_key = os.getenv("PRIVATE_KEY")  # From key symbol next to account
        
    def __str__(self):
        info = {'contract_location': self.contract_location,
                'compiled_contract_location': self.compiled_contract_location
               }
        return str(info)
    def compileContract(self):
        print('Compiling Contract...')
        with open(self.contract_location, "r") as file:
            hash_storage_file = file.read()

        # Compile our solidity file
        compiled_sol = compile_standard(
            {
                "language": "Solidity",
                "sources": {"HashStorage.sol": {"content": hash_storage_file}},
                "settings": {
                    "outputSelection": {
                        "*": {"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]}
                    }
                },
            },
            solc_version="0.6.0",
        )
        print('Contract Compiled!')
        print('\nWriting bytecode to JSON file...')
        with open(self.compiled_contract_location, "w") as file:
            json.dump(compiled_sol, file)
        print('File completed at {}'.format(self.compiled_contract_location))
        # Deploy file Prereqs
        # Get bytecode
        self.bytecode = compiled_sol["contracts"]["HashStorage.sol"]["HashStorage"]["evm"]["bytecode"]["object"]

        # get abi
        self.abi = compiled_sol["contracts"]["HashStorage.sol"]["HashStorage"]["abi"]
        
    def deployContract(self):
        self.compileContract()
        
        # Now we have all parameters we need to interact with Ganache local chain
        HashStorage = self.w3.eth.contract(abi=self.abi, bytecode=self.bytecode)  # This is our contract
        # Get latest transaction:
        nonce = self.w3.eth.getTransactionCount(
            self.my_address
        )  # gives our nonce - number of transactions
        # print(nonce)
        # 1. Build a transacton
        # 2. Sign a transaction
        # 3. Send a transaction
        transaction = HashStorage.constructor().buildTransaction(
            {
                "gasPrice": self.w3.eth.gas_price,
                "chainId": self.chain_id,
                "from": self.my_address,
                "nonce": nonce,
            }
        )
        signed_txn = self.w3.eth.account.sign_transaction(transaction, private_key=self.private_key)
        # send this signed transaction
        print("\nDeploying contract...")
        tx_hash = self.w3.eth.send_raw_transaction(
            signed_txn.rawTransaction
        )  # Can now view this in 'transactions' in ganache
        # Wait for block confirmation:
        tx_receipt = self.w3.eth.wait_for_transaction_receipt(tx_hash)
        # Get address of smart contract:
        contract_address = tx_receipt.contractAddress
        print("Deployed!")
        # Update contract address in .env file:
        file_lines = []
        with open('.env', 'r') as file:
            for line in file:
                if 'contract_address' not in line.lower():
                    file_lines.append(line)
        with open('.env', 'w') as file:
            for line in file_lines:
                file.write(line)
            file.write('contract_address=' + contract_address)
            file.close()
    

In [ ]:
contract = ContractCreator('./HashStorage.sol')
# contract.compileContract()
contract.deployContract()